In [58]:
import json
import pandas as pd
import requests
import re

In [59]:
# Global variables
school_number = "675"

In [60]:
def make_initial_request(school_number: int) -> tuple:
    url = f"https://www.ratemyprofessors.com/search/professors/{school_number}?q=*"

    # Make a get request to the URL and store the response as string
    response = requests.get(url).text

    # From the response, parse the schoolID and responseCount

    # Regular expression pattern to match the schoolID
    school_id_pattern = r'"schoolID\\":\\"(.*?)\\",'

    # Regular expression pattern to match the resultCount
    result_count_pattern = r'"resultCount":(\d+),'
    
    # Regular expression pattern to match the school name
    school_name_pattern = r'"__typename":"School","name":"(.*?)"'

    # Search for the patterns in the string
    school_id_match = re.search(school_id_pattern, response)
    result_count_match = re.search(result_count_pattern, response)
    school_name_match = re.search(school_name_pattern, response)

    if school_id_match:
        school_id = school_id_match.group(1)
        print(f"School ID: {school_id}")
    else:
        print("No school ID found.")

    if result_count_match:
        result_count = result_count_match.group(1)
        result_count = int(result_count)
        print(f"Result Count: {result_count}")
    else:
        print("No result count found.")
    
    if school_name_match:
        school_name = school_name_match.group(1)
        print(f"School Name: {school_name}")
    else:
        print("No school name found.")
    
    return (school_id, result_count, school_name)

In [61]:
def get_teacher_data(school_number: str, school_id: str, result_count: int, chunk_size=1000):
    url = "https://www.ratemyprofessors.com/graphql"
    headers = {
        "Accept": "*/*",
        "Accept-Language": "en-US,en;q=0.9",
        "Authorization": "Basic dGVzdDp0ZXN0",
        "Connection": "keep-alive",
        "Content-Type": "application/json",
        "Cookie": "ccpa-notice-viewed-02=true",
        "Origin": "https://www.ratemyprofessors.com",
        "Referer": f"https://www.ratemyprofessors.com/search/professors/{school_number}?q=*",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
        "sec-ch-ua": "\"Google Chrome\";v=\"125\", \"Chromium\";v=\"125\", \"Not.A/Brand\";v=\"24\"",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "\"macOS\""
    }

    all_results = []
    has_next_page = True
    cursor = ""

    while has_next_page and len(all_results) < result_count:
        body = {
            "query": """
            query TeacherSearchPaginationQuery(
              $count: Int!
              $cursor: String
              $query: TeacherSearchQuery!
            ) {
              search: newSearch {
                ...TeacherSearchPagination_search_1jWD3d
              }
            }

            fragment TeacherSearchPagination_search_1jWD3d on newSearch {
              teachers(query: $query, first: $count, after: $cursor) {
                didFallback
                edges {
                  node {
                    ...TeacherCard_teacher
                    id
                    __typename
                  }
                }
                pageInfo {
                  hasNextPage
                  endCursor
                }
                resultCount
                filters {
                  field
                  options {
                    value
                    id
                  }
                }
              }
            }

            fragment TeacherCard_teacher on Teacher {
              id
              legacyId
              avgRating
              numRatings
              ...CardFeedback_teacher
              ...CardSchool_teacher
              ...CardName_teacher
              ...TeacherBookmark_teacher
            }

            fragment CardFeedback_teacher on Teacher {
              wouldTakeAgainPercent
              avgDifficulty
            }

            fragment CardSchool_teacher on Teacher {
              department
              school {
                name
                id
              }
            }

            fragment CardName_teacher on Teacher {
              firstName
              lastName
            }

            fragment TeacherBookmark_teacher on Teacher {
              id
              isSaved
            }
            """,
            "variables": {
                "count": min(chunk_size, result_count - len(all_results)),
                "cursor": cursor,
                "query": {
                    "text": "",
                    "schoolID": f"{school_id}",
                    "fallback": True,
                    "departmentID": None
                }
            }
        }

        response = requests.post(url, headers=headers, json=body)
        data = response.json()

        if 'data' in data and 'search' in data['data'] and 'teachers' in data['data']['search']:
            teachers_data = data['data']['search']['teachers']
            if teachers_data['edges']:
                all_results.extend([edge['node'] for edge in teachers_data['edges']])
                has_next_page = teachers_data['pageInfo']['hasNextPage']
                cursor = teachers_data['pageInfo']['endCursor']
            else:
                has_next_page = False
        else:
            has_next_page = False
            print("Error in response:", data)

        # Print status to monitor progress
        print(f"Fetched {len(all_results)} teachers so far...")

    return all_results[:result_count]


In [62]:
data = make_initial_request(school_number)

school_id, result_count, school_name = data

School ID: U2Nob29sLTY3NQ==
Result Count: 5833
School Name: New York University


In [63]:
# Make another request with the result count
data = get_teacher_data(school_number, school_id, result_count)

Fetched 1000 teachers so far...
Fetched 2000 teachers so far...
Fetched 3000 teachers so far...
Fetched 4000 teachers so far...
Fetched 5000 teachers so far...
Fetched 5833 teachers so far...


In [64]:
# Extract the required variables and create a DataFrame
parsed_data = []

for entry in data:
    full_name = f"{entry['firstName']} {entry['lastName']}"
    rating = entry['avgRating']
    department = entry['department']
    difficulty = entry['avgDifficulty']
    num_ratings = entry['numRatings']
    
    parsed_data.append({
        'Name': full_name,
        'Average Rating': rating,
        'Department': department,
        'Difficulty Rating': difficulty,
        'Number of Ratings': num_ratings
    })

df = pd.DataFrame(parsed_data)
df

,Name,Average Rating,Department,Difficulty Rating,Number of Ratings
0,Gillian Gallagher,0.0,Linguistics,0.0,0
1,Charley Layton,0.0,Theater,0.0,0
2,Ramy Abied,0.0,Psychology,0.0,0
3,Larry Wein,0.0,Television,0.0,0
4,Daniel McNelis,0.0,Biology,0.0,0
...,...,...,...,...,...
5828,Leslie Ferraro,0.0,Marketing,0.0,0
5829,Linwood Lewis,0.0,Social Work,0.0,0
5830,Seth Watter,4.0,Communication,4.0,1
5831,Alison Taylor,0.0,Business,0.0,0


In [65]:
# Keep only unique rows
df = df.drop_duplicates()

df

,Name,Average Rating,Department,Difficulty Rating,Number of Ratings
0,Gillian Gallagher,0.0,Linguistics,0.0,0
1,Charley Layton,0.0,Theater,0.0,0
2,Ramy Abied,0.0,Psychology,0.0,0
3,Larry Wein,0.0,Television,0.0,0
4,Daniel McNelis,0.0,Biology,0.0,0
...,...,...,...,...,...
5828,Leslie Ferraro,0.0,Marketing,0.0,0
5829,Linwood Lewis,0.0,Social Work,0.0,0
5830,Seth Watter,4.0,Communication,4.0,1
5831,Alison Taylor,0.0,Business,0.0,0


In [66]:
# Remove rows with Number of Ratings of 0
df = df[df['Number of Ratings'] != 0]

df

,Name,Average Rating,Department,Difficulty Rating,Number of Ratings
17,Pat Rock,4.6,Humanities,3.4,5
18,Linda Gordon,4.8,History,2.5,2
19,Margarita Krochik,1.0,Psychology,5.0,1
20,Olivier Pauluis,1.8,Mathematics,3.6,11
21,Maeve Adams,3.7,English,3.8,13
...,...,...,...,...,...
5822,Lorenzo Castellano,5.0,Anthropology,2.0,1
5823,Nikolay Kukushkin,4.3,Liberal Studies,3.6,63
5825,Anasse Bari,4.3,Computer Science,3.6,209
5830,Seth Watter,4.0,Communication,4.0,1


In [67]:
# Get mean rating for each department
mean_ratings = df.groupby('Department')['Average Rating'].mean().reset_index()
mean_ratings = mean_ratings.sort_values(by='Average Rating', ascending=False)

mean_ratings

,Department,Average Rating
116,Paleontology,5.00
107,Natural Sciences,5.00
20,Arts amp Sciences,5.00
65,Genetics,5.00
115,Occupational Therapy,5.00
...,...,...
5,American Studies,2.80
122,Professional Programs,2.55
27,Business Law,2.50
142,Teaching Learning,2.00


In [68]:
# Get mean difficulty rating for each department
mean_difficulty = df.groupby('Department')['Difficulty Rating'].mean().reset_index()
mean_difficulty = mean_difficulty.sort_values(by='Difficulty Rating', ascending=False)

mean_difficulty

,Department,Difficulty Rating
142,Teaching Learning,4.500000
9,Architecture,4.500000
83,International Affairs,4.333333
60,Foundations of Am. Culture/Classics,4.300000
81,Interactive Telecommunications,4.250000
...,...,...
127,Religion,2.000000
115,Occupational Therapy,2.000000
3,African Studies,1.916667
144,Teaching amp Learning,1.650000


In [69]:
# Get number of ratings for each department
num_ratings = df.groupby('Department')['Number of Ratings'].sum().reset_index()
num_ratings = num_ratings.sort_values(by='Number of Ratings', ascending=False)

num_ratings

,Department,Number of Ratings
152,Writing,5128
100,Mathematics,3584
42,Economics,3001
33,Computer Science,2814
123,Psychology,2532
...,...,...
137,Speech amp Hearing Sciences,2
1,Administration,2
20,Arts amp Sciences,2
65,Genetics,1


In [70]:
# Create a matrix of the mean ratings, mean difficulty ratings, and number of ratings for each department
department_stats = mean_ratings.merge(mean_difficulty, on='Department')
department_stats = department_stats.merge(num_ratings, on='Department')

# Replace ' amp ' with ' & ' in the Department column
department_stats['Department'] = department_stats['Department'].str.replace(' amp ', ' & ')

department_stats

,Department,Average Rating,Difficulty Rating,Number of Ratings
0,Paleontology,5.00,3.600000,585
1,Natural Sciences,5.00,3.100000,99
2,Arts & Sciences,5.00,2.500000,2
3,Genetics,5.00,1.000000,1
4,Occupational Therapy,5.00,2.000000,1
...,...,...,...,...
148,American Studies,2.80,3.550000,23
149,Professional Programs,2.55,3.208333,35
150,Business Law,2.50,2.700000,7
151,Teaching Learning,2.00,4.500000,6


In [71]:
# Get departments with 50 or more ratings
department_stats = department_stats[department_stats['Number of Ratings'] >= 50]

department_stats

,Department,Average Rating,Difficulty Rating,Number of Ratings
0,Paleontology,5.000000,3.600000,585
1,Natural Sciences,5.000000,3.100000,99
11,Independent Studies,4.800000,3.450000,109
16,Foreign Languages,4.600000,2.935000,63
17,German,4.547368,2.905263,72
...,...,...,...,...
135,Nursing,3.291304,3.078261,288
136,Hospitality,3.268000,2.956000,150
138,Expository Writing,3.180000,3.200000,50
145,Foundations of Am. Culture/Classics,2.900000,4.300000,168


In [72]:
# Save the df as a json file
df.to_json('nyu-prof-data.json', orient='records')